Esta guía detalla los comandos exactos utilizados para instalar los servicios de Docker, n8n, grafana y PostgreSQL en la Raspberry Pi.

Nota técnica: Durante la carga de los servicios, surgió un error debido a la demora en la respuesta del sistema. Esto se solucionó reiniciando la Raspberry Pi y permitiendo que se enfriara, ya que se detectó un exceso de temperatura.
1. Instalación de Docker y Dependencias

In [ ]:
# Actualizar el sistema
sudo apt-get update && sudo apt-get upgrade -y
# Instalar Docker usando el script oficial
curl -sSL [https://get.docker.com](https://get.docker.com) | sh
# Dar permisos al usuario actual (pasty)
sudo usermod -aG docker $USER
# Instalar dependencias y Docker Compose Plugin
sudo apt-get install -y libffi-dev libssl-dev python3 python3-pip
sudo apt-get install -y docker-compose-plugin

A continuación, creamos la carpeta del proyecto iot_project

In [ ]:
services:
  db:
    image: postgres:15
    container_name: postgres_pi
    restart: always
    environment:
      POSTGRES_USER: admin
      POSTGRES_PASSWORD: tu_password_segura  
      POSTGRES_DB: iot_db
    volumes:
      - ./postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

  n8n:
    image: n8nio/n8n:latest
    container_name: n8n_pi
    restart: always
    ports:
      - "5678:5678"
    environment:
      - N8N_HOST=192.168.100.53 # O raspberrypi.local
      - N8N_SECURE_COOKIE=false
    volumes:
      - ./n8n_data:/home/node/.n8n
    depends_on:
      - db

  grafana:
    image: grafana/grafana:latest
    container_name: grafana_pi
    restart: always
    ports:
      - "3000:3000"
    depends_on:
      - db

In [ ]:
# Crear carpeta del proyecto
mkdir ~/iot_project && cd ~/iot_project

# Crear carpetas de datos
mkdir n8n_data postgres_data

# Aplicar permisos totales para evitar bloqueos del contenedor
sudo chmod -R 777 ~/iot_project/n8n_data

Al ejecutar n8n e intentar acceder desde Chrome para poder crear una cuenta me mando un error, por lo que fue necesario modificar el archivo de configuración dentro de iot_project (el código compartido ya incluye este cambio), añadiendo la línea - N8N_SECURE_COOKIE=false en la sección de enviroment

In [ ]:
# Levantar todos los servicios en segundo plano
docker compose up -d

# Si n8n requiere un reinicio limpio tras cambiar el N8N_SECURE_COOKIE:
docker compose stop n8n
docker compose rm -f n8n
docker compose up -d n8n